In [1]:
import numpy as np;
import pandas as pd;

## Initialize Parameters

In [2]:
## Parameters and Global variables
district = "tvm"


## Load and Initialize Data

In [3]:
## Population data
initDataDF = pd.read_csv("../data/" + district + "_init_data.csv")

## Number of regions
r = len(initDataDF.index)

## T-Matrix
tData = pd.read_csv("../output/" + district + "_tmatrix_results.csv")

## Sort distance data by name for easy matrix transformation
initDataDF.sort_values(by=['name'], inplace=True)

In [4]:
tData.head(5)
# TODO fix the unnamed colum

,Unnamed: 0,iName,jName,Tij
0,0,Amboori__Thiruvananthapuram,Amboori__Thiruvananthapuram,15283.200000
1,1,Amboori__Thiruvananthapuram,Anad__Thiruvananthapuram,2.632907
2,2,Amboori__Thiruvananthapuram,Anchuthengu__Thiruvananthapuram,0.334459
3,3,Amboori__Thiruvananthapuram,Andoorkonam__Thiruvananthapuram,1.126357
4,4,Amboori__Thiruvananthapuram,Aruvikkara__Thiruvananthapuram,4.373778


## Prepare T-Matrix

In [5]:

tMatrix = tData['Tij'].to_numpy().reshape(r,r)


## Function Definition

In [6]:
def getCijv2(i, j, tMatrix, Ni):
    cwSum = 0
    for k in range(0,r-1):
        Tik = tMatrix[i][k]
        Tjk = tMatrix[j][k]
        TlkSum = tMatrix.sum(axis=0)[k]
        
        cwSum = cwSum + (Tik/Ni)*(Tjk/TlkSum)

    return cwSum

In [7]:
def getCij(i, j, tMatrix, initDataDF):
    Ni = initDataDF.iloc[i]['S']
    cwSum = 0
    for k in range(0,r-1):
        Tik = tMatrix[i][k]
        Tjk = tMatrix[j][k]
        TlkSum = 0;

        for l in range(0, r-1):
            TlkSum = TlkSum + tMatrix[l][k]
        
        cwSum = cwSum + (Tik/Ni)*(Tjk/TlkSum)

    return cwSum

In [8]:
getCij(0,1, tMatrix, initDataDF)

0.00031567593139767813

## Compute C-Matrix

In [9]:
data = []
for i in range(0,r):
    Ni = initDataDF.iloc[i]['S']
    for j in range(0,r):
        Cij = getCijv2(i,j,tMatrix, Ni)
        data.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Cij': Cij})

resultsDF = pd.DataFrame(data);


## Save Output

In [10]:
resultsDF.to_csv("../output/" + district + "_cmatrix_results.csv")

In [11]:
resultsDF.head(10)

,iName,jName,Cij
0,Amboori__Thiruvananthapuram,Amboori__Thiruvananthapuram,0.932860
1,Amboori__Thiruvananthapuram,Anad__Thiruvananthapuram,0.000316
2,Amboori__Thiruvananthapuram,Anchuthengu__Thiruvananthapuram,0.000045
3,Amboori__Thiruvananthapuram,Andoorkonam__Thiruvananthapuram,0.000148
4,Amboori__Thiruvananthapuram,Aruvikkara__Thiruvananthapuram,0.000460
5,Amboori__Thiruvananthapuram,Aryanad__Thiruvananthapuram,0.000809
6,Amboori__Thiruvananthapuram,Aryancodu__Thiruvananthapuram,0.003573
7,Amboori__Thiruvananthapuram,Athiyannoor__Thiruvananthapuram,0.000449
8,Amboori__Thiruvananthapuram,Attingal(M)__Thiruvananthapuram,0.000185
9,Amboori__Thiruvananthapuram,Azhoor__Thiruvananthapuram,0.000106
